In [2]:
import numpy as np
import pandas as pd
from rpy2.robjects.packages import importr
stats = importr('stats')

from fforma.meta_model import MetaModels
from fforma.base_models import Naive2
from fforma.base_models_r import (
    ARIMA, 
    ETS, 
    NNETAR,
    TBATS,
    STLM,
    RandomWalk,
    ThetaF, 
    Naive, 
    SeasonalNaive
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
u_id = np.array(['uid_1', 'uid_2', 'uid_3', 'uid_4'])

horizons = np.array([1, 2, 3, 4])
season = [1, 2, 1, 2]

y = np.array([[1.1, 1.2, 1.3, 1.4, 1.5],
              [2.1, 2.2, 2.3, 2.4, 2.5],
              [3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7],
              [4.1, 4.2, 4.3, 4.4, 4.5]])

fforma_df = pd.DataFrame({'unique_id': u_id,
                   'seasonality': season,
                   'y': y})

df_horizon = pd.DataFrame({'unique_id': u_id, 'horizon': horizons})

In [14]:
fforma_df

,unique_id,seasonality,y
0,uid_1,1,"[1.1, 1.2, 1.3, 1.4, 1.5]"
1,uid_2,2,"[2.1, 2.2, 2.3, 2.4, 2.5]"
2,uid_3,1,"[3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7]"
3,uid_4,2,"[4.1, 4.2, 4.3, 4.4, 4.5]"


In [15]:
df_horizon

,unique_id,horizon
0,uid_1,1
1,uid_2,2
2,uid_3,3
3,uid_4,4


In [16]:
meta_models = {
     'ARIMA': lambda freq: ARIMA(freq, stepwise=False, approximation=False),
     'ETS': ETS,
     'NNETAR': NNETAR,
     'TBATS': TBATS,
     'STLM': lambda freq: ARIMA(freq, d=0, D=0) if freq == 1 else STLM(freq, modelfunction=stats.ar),
     'RandomWalkDrift': lambda freq: RandomWalk(freq=freq, drift=True),
     'ThetaF': ThetaF,
     'Naive': Naive,
     'SeasonalNaive': SeasonalNaive,
     'Naive2': Naive2,
}

In [18]:
model = MetaModels(meta_models).fit(fforma_df)

[########################################] | 100% Completed | 14.8s


In [19]:
df = model.predict(df_horizon)

[########################################] | 100% Completed | 14.1s


In [20]:
df

,unique_id,ARIMA,ETS,NNETAR,Naive,Naive2,RandomWalkDrift,STLM,SeasonalNaive,TBATS,ThetaF
0,uid_1,[1.5999999999999999],[1.5999999999999999],[1.5737559736074875],[1.6],[1.2999999999981156],[1.6],[1.55],[1.5],[1.5],[1.5]
1,uid_2,"[2.5999999999999996, 2.6999999999999997]","[2.6, 2.7]","[2.551944255553706, 2.5704350620020544]","[2.599999999999999, 2.6999999999999993]","[2.299996952974814, 2.300004858155279]","[2.6, 2.7]","[2.5500000000000003, 2.6]","[2.5, 2.5]","[2.4, 2.5]","[2.5, 2.5]"
2,uid_3,"[3.8000000000000003, 3.9000000000000004, 4.0]","[3.8000000000000003, 3.9000000000000004, 4.0]","[3.7789764043217793, 3.8341625143064406, 3.867...","[3.7999999999999994, 3.899999999999999, 3.9999...","[3.6787678027398534, 3.659038292814686, 3.6407...","[3.8000000000000003, 3.9000000000000004, 4.0]","[3.749994998453236, 3.799994998453236, 3.84999...","[3.7, 3.7, 3.7]","[3.7, 3.7, 3.7]","[3.7, 3.7, 3.7]"
3,uid_4,"[4.6, 4.7, 4.800000000000001, 4.9]","[4.6000000000000005, 4.700000000000001, 4.8000...","[4.550312816053603, 4.566934455229994, 4.57150...","[4.600000000000001, 4.700000000000002, 4.80000...","[4.299996952974814, 4.30000485815528, 4.299996...","[4.6, 4.7, 4.800000000000001, 4.9]","[4.55, 4.6000000000000005, 4.65, 4.70000000000...","[4.5, 4.5, 4.5, 4.5]","[4.4, 4.5, 4.4, 4.5]","[4.5, 4.5, 4.5, 4.5]"
